In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from scipy import sparse
import string
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import preprocessing, model_selection
import time
import re
from datetime import datetime
# from scipy.stats import boxcox
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# data = pd.read_csv("../input/sample_submission.csv")
train = pd.read_json(open("../input/train.json", "r"))
test = pd.read_json(open("../input/test.json", "r"))
print 'train shape:',train.shape
print 'test shape:',test.shape

y_map = {'low': 2, 'medium': 1, 'high': 0}
target = train['interest_level'].apply(lambda x: y_map[x]).values

# ntrain = train.shape[0]

train shape: (49352, 15)
test shape: (74659, 14)


# Price

In [3]:
tmp = pd.concat([train['price'],test['price']])
ulimit = np.percentile(tmp.values, 99)
tmp.ix[tmp>ulimit] = ulimit

train['price'].ix[train['price']>ulimit] = ulimit
train.loc[:,'sc_price'] = (train['price'] - tmp.mean()) / tmp.std()

test['price'].ix[test['price']>ulimit] = ulimit
test.loc[:,'sc_price'] = (test['price'] - tmp.mean()) / tmp.std()


/home/xujin/AI/anaconda2/lib/python2.7/site-packages/pandas/core/indexing.py:140: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


# Rooms

In [4]:
tmp = pd.concat([train['bathrooms'],test['bathrooms']])
ulimit = 5
tmp.ix[tmp> ulimit] = ulimit


train['bathrooms'].ix[train['bathrooms']>ulimit] = ulimit
train.loc[:,'sc_bathrooms'] = (train['bathrooms'] - tmp.mean()) / tmp.std()

test['bathrooms'].ix[test['bathrooms']>ulimit] = ulimit
test.loc[:,'sc_bathrooms'] = (test['bathrooms'] - tmp.mean()) / tmp.std()


In [5]:
tmp = pd.concat([train['bedrooms'],test['bedrooms']])
ulimit = 7
tmp.ix[tmp> ulimit] = ulimit


train['bedrooms'].ix[train['bedrooms']>ulimit] = ulimit
train.loc[:,'sc_bedrooms'] = (train['bedrooms'] - tmp.mean()) / tmp.std()

test['bedrooms'].ix[test['bedrooms']>ulimit] = ulimit
test.loc[:,'sc_bedrooms'] = (test['bedrooms'] - tmp.mean()) / tmp.std()

# Longitude & Latitude

In [6]:
tmp = pd.concat([train['longitude'],test['longitude']])
llimit = np.percentile(tmp.values, 1)
ulimit = np.percentile(tmp.values, 99)
tmp.ix[tmp > ulimit] = ulimit
tmp.ix[tmp < llimit] = llimit


train['longitude'].ix[train['longitude']>ulimit] = ulimit
train['longitude'].ix[train['longitude']<llimit] = llimit
train.loc[:,'sc_longitude'] = (train['longitude'] - tmp.mean()) / tmp.std()

test['longitude'].ix[test['longitude']>ulimit] = ulimit
test['longitude'].ix[test['longitude']<llimit] = llimit
test.loc[:,'sc_longitude'] = (test['longitude'] - tmp.mean()) / tmp.std()

In [7]:
tmp = pd.concat([train['latitude'],test['latitude']])
llimit = np.percentile(tmp.values, 1)
ulimit = np.percentile(tmp.values, 99)
tmp.ix[tmp > ulimit] = ulimit
tmp.ix[tmp < llimit] = llimit


train['latitude'].ix[train['latitude']>ulimit] = ulimit
train['latitude'].ix[train['latitude']<llimit] = llimit
train.loc[:,'sc_latitude'] = (train['latitude'] - tmp.mean()) / tmp.std()

test['latitude'].ix[test['latitude']>ulimit] = ulimit
test['latitude'].ix[test['latitude']<llimit] = llimit
test.loc[:,'sc_latitude'] = (test['latitude'] - tmp.mean()) / tmp.std()

In [8]:
features_to_use = ['sc_price','sc_bathrooms','sc_bedrooms','sc_longitude', 'sc_latitude']

# Categoricals

In [9]:
lbl = preprocessing.LabelEncoder()
categorical = ["display_address", "manager_id", "building_id", "street_address"]
for f in categorical:
        if train[f].dtype=='object':
            lbl.fit(list(train[f].values) + list(test[f].values))
            train[f+'_lbl'] = lbl.transform(list(train[f].values))
            test[f+'_lbl'] = lbl.transform(list(test[f].values))
            features_to_use.append(f+'_lbl')

# Date

In [10]:
train["created"] = pd.to_datetime(train["created"])
test["created"] = pd.to_datetime(test["created"])

train["created_month"] = train["created"].dt.month
test["created_month"] = test["created"].dt.month

train['created_weekday'] = train["created"].dt.weekday
test['created_weekday'] = test["created"].dt.weekday

train["created_day"] = train["created"].dt.day
test["created_day"] = test["created"].dt.day

train['created_hour'] = train["created"].dt.hour
test['created_hour'] = test["created"].dt.hour


features_to_use.extend(["created_month",'created_weekday',"created_day",'created_hour'])

# Description & Feature

In [11]:
# count of "features" #
train["num_features"] = train["features"].apply(len)
test["num_features"] = test["features"].apply(len)

# count of words present in description column #
train['description'] = train['description'].apply(lambda x: x.replace('<p><a  website_redacted ', ''))
train['description'] = train['description'].apply(lambda x: x.replace('!<br /><br />', ''))
test['description'] = test['description'].apply(lambda x: x.replace('<p><a  website_redacted ', ''))
test['description'] = test['description'].apply(lambda x: x.replace('!<br /><br />', ''))


string.punctuation.__add__('!!')
string.punctuation.__add__('(')
string.punctuation.__add__(')')

remove_punct_map = dict.fromkeys(map(ord, string.punctuation))

train['description'] = train['description'].apply(lambda x: x.translate(remove_punct_map))
test['description'] = test['description'].apply(lambda x: x.translate(remove_punct_map))

train['num_description_letter'] = train['description'].apply(lambda x: len(x.strip()))
train['num_description_words'] = train['description'].apply(lambda x: 0 if len(x.strip()) == 0 else len(x.split(' ')))
test['num_description_letter'] = test['description'].apply(lambda x: len(x.strip()))
test['num_description_words'] = test['description'].apply(lambda x: 0 if len(x.strip()) == 0 else len(x.split(' ')))


features_to_use.extend(["num_features","num_description_words", 'num_description_letter'])

In [12]:
train['features'] = train["features"].apply(lambda x: " ".join(["_".join(i.split(" ")) for i in x]))
test['features'] = test["features"].apply(lambda x: " ".join(["_".join(i.split(" ")) for i in x]))

In [13]:
num = 200

c_vect = CountVectorizer(stop_words='english', max_features=num)
c_vect.fit(pd.concat([train['features'],test['features']]))
tr_sparse = c_vect.fit_transform(train["features"])
te_sparse = c_vect.transform(test["features"])
sparse_cols = c_vect.get_feature_names()

In [14]:
# train_test['desc'] = train_test['description']
# train_test['desc'] = train_test['desc'].apply(lambda x: x.replace('<p><a  website_redacted ', ''))
# train_test['desc'] = train_test['desc'].apply(lambda x: x.replace('!<br /><br />', ''))

# string.punctuation.__add__('!!')
# string.punctuation.__add__('(')
# string.punctuation.__add__(')')

# remove_punct_map = dict.fromkeys(map(ord, string.punctuation))

# train_test['desc'] = train_test['desc'].apply(lambda x: x.translate(remove_punct_map))
# train_test['desc_letters_count'] = train_test['description'].apply(lambda x: len(x.strip()))
# train_test['desc_words_count'] = train_test['desc'].apply(lambda x: 0 if len(x.strip()) == 0 else len(x.split(' ')))

# train_test.drop(['description', 'desc'], axis=1, inplace=True)
# print 'train_test shape: ', train_test.shape

# train_test['features_count'] = train_test['features'].apply(lambda x: len(x))
# train_test['features2'] = train_test['features']
# train_test['features2'] = train_test['features2'].apply(lambda x: ' '.join(x))

# c_vect = CountVectorizer(stop_words='english', max_features=200, ngram_range=(1, 1))
# c_vect.fit(train_test['features2'])

# c_vect_sparse_1 = c_vect.transform(train_test['features2'])
# c_vect_sparse1_cols = c_vect.get_feature_names()



# train_test.drop(['features', 'features2'], axis=1, inplace=True)
# print 'train_test shape: ', train_test.shape

In [15]:
# train_test['desc'] = train_test['description']
# train_test['desc'] = train_test['desc'].apply(lambda x: x.replace('<p><a  website_redacted ', ''))
# train_test['desc'] = train_test['desc'].apply(lambda x: x.replace('!<br /><br />', ''))

# string.punctuation.__add__('!!')
# string.punctuation.__add__('(')
# string.punctuation.__add__(')')

# remove_punct_map = dict.fromkeys(map(ord, string.punctuation))

# train_test['desc'] = train_test['desc'].apply(lambda x: x.translate(remove_punct_map))


# train_test['features']=train_test['features'].apply(lambda x: ', '.join(x))
# import nltk
# from nltk.tag import pos_tag
# from nltk import word_tokenize
# from nltk.corpus import stopwords
# stop = stopwords.words('english')
# def cleaning_text(sentence):
#    sentence=sentence.lower()
#    sentence=re.sub('[^\w\s]',' ', sentence) #removes punctuations
#    sentence=re.sub('\d+',' ', sentence) #removes digits
#    cleaned=' '.join([w for w in sentence.split() if not w in stop]) # removes english stopwords
#    cleaned=' '.join([w for w , pos in pos_tag(cleaned.split()) if (pos == 'NN' or pos=='JJ' or pos=='JJR' or pos=='JJS' )])
#    #selecting only nouns and adjectives
#    cleaned=' '.join([w for w in cleaned.split() if not len(w)<=2 ]) #removes single lettered words and digits
#    cleaned=cleaned.strip()
#    return cleaned
 
# train_test['desc_cleaned']= train_test['desc'].apply(lambda x: cleaning_text(x))
# train_test['feat_cleaned']= train_test['features'].apply(lambda x: cleaning_text(x))
# train_test["final_feat"] = train_test["desc_cleaned"].map(str) +" "+train_test["feat_cleaned"]

# c_vect = CountVectorizer(stop_words='english', max_features=200, ngram_range=(1, 1))
# c_vect.fit(train_test['final_feat'])

# c_vect_sparse_1 = c_vect.transform(train_test['final_feat'])
# c_vect_sparse1_cols = c_vect.get_feature_names()


# train_test.drop(['description', 'desc','desc_cleaned'], axis=1, inplace=True)
# train_test.drop(['features','feat_cleaned','final_feat'], axis=1, inplace=True)
# print 'train_test shape: ', train_test.shape

# Address

In [16]:
train['address1'] = train['display_address']
train['address1'] = train['address1'].apply(lambda x: x.lower())

test['address1'] = test['display_address']
test['address1'] = test['address1'].apply(lambda x: x.lower())

address_map = {
    'w': 'west',
    'st.': 'street',
    'ave': 'avenue',
    'st': 'street',
    'e': 'east',
    'n': 'north',
    's': 'south'
}


def address_map_func(s):
    s = s.split(' ')
    out = []
    for x in s:
        if x in address_map:
            out.append(address_map[x])
        else:
            out.append(x)
    return ' '.join(out)


train['address1'] = train['address1'].apply(lambda x: x.translate(remove_punct_map))
test['address1'] = test['address1'].apply(lambda x: address_map_func(x))

new_cols = ['street', 'avenue', 'east', 'west', 'north', 'south']

for col in new_cols:
    train[col] = train['address1'].apply(lambda x: 1 if col in x else 0)
    test[col] = test['address1'].apply(lambda x: 1 if col in x else 0)

train['other_address'] = train[new_cols].apply(lambda x: 1 if x.sum() == 0 else 0, axis=1)
test['other_address'] = test[new_cols].apply(lambda x: 1 if x.sum() == 0 else 0, axis=1)



features_to_use.extend(['street', 'avenue', 'east', 'west', 'north', 'south','other_address'])

# Photo

In [17]:
# count of photos #
train["num_photos"] = train["photos"].apply(len)
test["num_photos"] = test["photos"].apply(len)

features_to_use.append("num_photos")

In [18]:
# train_test['photos_count'] = train_test['photos'].apply(lambda x: len(x))
# train_test.drop(['photos', 'display_address', 'street_address'], axis=1, inplace=True)
# print 'train_test shape: ', train_test.shape

# Manager_id

In [19]:

temp = pd.concat([train.manager_id_lbl,pd.get_dummies(train['interest_level'])], axis = 1).groupby('manager_id_lbl').mean()
temp.columns = ['high_frac','low_frac', 'medium_frac']
temp['count'] = train.groupby('manager_id_lbl').count().iloc[:,1]

In [20]:
# compute skill
temp['manager_skill'] = temp['high_frac']*2 + temp['medium_frac']

# get ixes for unranked managers...
unranked_managers_ixes = temp['count']<20
# ... and ranked ones
ranked_managers_ixes = ~unranked_managers_ixes

# compute mean values from ranked managers and assign them to unranked ones
mean_values = temp.loc[~unranked_managers_ixes, ['high_frac','low_frac', 'medium_frac','manager_skill']].mean()
# print(mean_values)
mean_count = np.mean(temp.loc[unranked_managers_ixes,'count'])
temp.loc[unranked_managers_ixes,['high_frac','low_frac', 'medium_frac','manager_skill']] = mean_values.values
# print(temp.tail(10))

In [21]:
# inner join to assign manager features to the managers in the training dataframe
train = train.merge(temp.reset_index(),how='left', left_on='manager_id_lbl', right_on='manager_id_lbl')
# train.head()

In [22]:
features_to_use.extend(['high_frac','low_frac', 'medium_frac','manager_skill','count'])

In [23]:
# inner join to assign manager features to the managers in the testing dataframe
test = test.merge(temp.reset_index(),how='left', left_on='manager_id_lbl', right_on='manager_id_lbl')
index = test['manager_skill'].index[test['manager_skill'].apply(np.isnan)]
test.loc[index,['high_frac','low_frac', 'medium_frac','manager_skill']] = mean_values.values
test.loc[index,'count'] = int(mean_count)
# test.head()

In [24]:
# manager = train[['manager_id','interest_level']]
# interest_dummies = pd.get_dummies(manager.interest_level)
# manager = pd.concat([manager,interest_dummies[['low','medium','high']]],
#                     axis = 1).drop('interest_level', axis = 1)

# manager_socre = pd.concat([manager.groupby('manager_id',as_index=False).mean(),
#                            manager.groupby('manager_id',as_index=False).count()], 
#                           axis = 1).iloc[:,[0,1,2,3,5]]
# manager_socre.columns = ['manager_id','low','medium','high','count']
# manager_socre['manager_score'] = manager_socre['medium']*1 + manager_socre['low']*2 


# train_test = train_test.merge(manager_socre[['manager_id','manager_score']],how = 'left')
# print 'train_test shape: ', train_test.shape

In [25]:
# managers_count = train_test['manager_id'].value_counts()

# train_test['top_10_manager'] = train_test['manager_id'].apply(lambda x: 1 if x in managers_count.index.values[
#     managers_count.values >= np.percentile(managers_count.values, 90)] else 0)
# train_test['top_25_manager'] = train_test['manager_id'].apply(lambda x: 1 if x in managers_count.index.values[
#     managers_count.values >= np.percentile(managers_count.values, 75)] else 0)
# train_test['top_5_manager'] = train_test['manager_id'].apply(lambda x: 1 if x in managers_count.index.values[
#     managers_count.values >= np.percentile(managers_count.values, 95)] else 0)
# train_test['top_50_manager'] = train_test['manager_id'].apply(lambda x: 1 if x in managers_count.index.values[
#     managers_count.values >= np.percentile(managers_count.values, 50)] else 0)
# train_test['top_1_manager'] = train_test['manager_id'].apply(lambda x: 1 if x in managers_count.index.values[
#     managers_count.values >= np.percentile(managers_count.values, 99)] else 0)
# train_test['top_2_manager'] = train_test['manager_id'].apply(lambda x: 1 if x in managers_count.index.values[
#     managers_count.values >= np.percentile(managers_count.values, 98)] else 0)
# train_test['top_15_manager'] = train_test['manager_id'].apply(lambda x: 1 if x in managers_count.index.values[
#     managers_count.values >= np.percentile(managers_count.values, 85)] else 0)
# train_test['top_20_manager'] = train_test['manager_id'].apply(lambda x: 1 if x in managers_count.index.values[
#     managers_count.values >= np.percentile(managers_count.values, 80)] else 0)
# train_test['top_30_manager'] = train_test['manager_id'].apply(lambda x: 1 if x in managers_count.index.values[
#     managers_count.values >= np.percentile(managers_count.values, 70)] else 0)

# Building_id

In [26]:
# train_test['Zero_building_id'] = train_test['building_id'].apply(lambda x: 1 if x == '0' else 0)

In [27]:
# buildings_count = train_test['building_id'].value_counts()

# train_test['top_10_building'] = train_test['building_id'].apply(lambda x: 1 if x in buildings_count.index.values[
#     buildings_count.values >= np.percentile(buildings_count.values, 90)] else 0)
# train_test['top_25_building'] = train_test['building_id'].apply(lambda x: 1 if x in buildings_count.index.values[
#     buildings_count.values >= np.percentile(buildings_count.values, 75)] else 0)
# train_test['top_5_building'] = train_test['building_id'].apply(lambda x: 1 if x in buildings_count.index.values[
#     buildings_count.values >= np.percentile(buildings_count.values, 95)] else 0)
# train_test['top_50_building'] = train_test['building_id'].apply(lambda x: 1 if x in buildings_count.index.values[
#     buildings_count.values >= np.percentile(buildings_count.values, 50)] else 0)
# train_test['top_1_building'] = train_test['building_id'].apply(lambda x: 1 if x in buildings_count.index.values[
#     buildings_count.values >= np.percentile(buildings_count.values, 99)] else 0)
# train_test['top_2_building'] = train_test['building_id'].apply(lambda x: 1 if x in buildings_count.index.values[
#     buildings_count.values >= np.percentile(buildings_count.values, 98)] else 0)
# train_test['top_15_building'] = train_test['building_id'].apply(lambda x: 1 if x in buildings_count.index.values[
#     buildings_count.values >= np.percentile(buildings_count.values, 85)] else 0)
# train_test['top_20_building'] = train_test['building_id'].apply(lambda x: 1 if x in buildings_count.index.values[
#     buildings_count.values >= np.percentile(buildings_count.values, 80)] else 0)
# train_test['top_30_building'] = train_test['building_id'].apply(lambda x: 1 if x in buildings_count.index.values[
#     buildings_count.values >= np.percentile(buildings_count.values, 70)] else 0)


In [28]:
# build = train[['building_id','interest_level']]
# interest_dummies = pd.get_dummies(build.interest_level)
# build = pd.concat([build,interest_dummies[['low','medium','high']]],
#                   axis = 1).drop('interest_level', axis = 1)

# build_socre = pd.concat([build.groupby('building_id',as_index=False).mean(),
#                          build.groupby('building_id',as_index=False).count()], 
#                         axis = 1).iloc[:,[0,1,2,3,5]]
# build_socre.columns = ['building_id','low','medium','high','count']
# build_socre['building_score'] = build_socre['medium']*1 + build_socre['low']*2 



# train_test = train_test.merge(build_socre[['building_id','building_score']],how = 'left')
# print 'train_test shape: ', train_test.shape

In [29]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49352 entries, 0 to 49351
Data columns (total 45 columns):
bathrooms                 49352 non-null float64
bedrooms                  49352 non-null int64
building_id               49352 non-null object
created                   49352 non-null datetime64[ns]
description               49352 non-null object
display_address           49352 non-null object
features                  49352 non-null object
interest_level            49352 non-null object
latitude                  49352 non-null float64
listing_id                49352 non-null int64
longitude                 49352 non-null float64
manager_id                49352 non-null object
photos                    49352 non-null object
price                     49352 non-null float64
street_address            49352 non-null object
sc_price                  49352 non-null float64
sc_bathrooms              49352 non-null float64
sc_bedrooms               49352 non-null float64
sc_longitude  

# Feature

In [30]:
features_to_use

['sc_price',
 'sc_bathrooms',
 'sc_bedrooms',
 'sc_longitude',
 'sc_latitude',
 'display_address_lbl',
 'manager_id_lbl',
 'building_id_lbl',
 'street_address_lbl',
 'created_month',
 'created_weekday',
 'created_day',
 'created_hour',
 'num_features',
 'num_description_words',
 'num_description_letter',
 'street',
 'avenue',
 'east',
 'west',
 'north',
 'south',
 'other_address',
 'num_photos',
 'high_frac',
 'low_frac',
 'medium_frac',
 'manager_skill',
 'count']

In [31]:
len(features_to_use)

29

In [32]:
train_X = sparse.hstack([train[features_to_use], tr_sparse]).tocsr()
test_X = sparse.hstack([test[features_to_use], te_sparse]).tocsr()

In [34]:
now = datetime.now()

name_feautres = '../input/featurestouse_' + str(now.strftime("%Y-%m-%d-%H-%M")) + '.pkl'
name_train_X = '../input/train_X_' + str(now.strftime("%Y-%m-%d-%H-%M")) + '.pkl'
name_test_X = '../input/test_X_' + str(now.strftime("%Y-%m-%d-%H-%M")) + '.pkl'
name_y = '../input/y_' + str(now.strftime("%Y-%m-%d-%H-%M")) + '.pkl'
name_sparse = '../input/sparse_' + str(now.strftime("%Y-%m-%d-%H-%M")) + '.pkl'
name_train = '../input/train_' + str(now.strftime("%Y-%m-%d-%H-%M")) + '.pkl'
name_test = '../input/test_' + str(now.strftime("%Y-%m-%d-%H-%M")) + '.pkl'

pd.to_pickle(features_to_use,name_feautres)
pd.to_pickle(train_X, name_train_X)
pd.to_pickle(test_X, name_test_X)
pd.to_pickle(target, name_y)
pd.to_pickle(sparse_cols, name_sparse)
pd.to_pickle(train, name_train)
pd.to_pickle(test, name_test)

name_tr_sparse = '../input/tr_sparse_' + str(now.strftime("%Y-%m-%d-%H-%M")) + '.pkl'
name_te_sparse = '../input/te_sparse_' + str(now.strftime("%Y-%m-%d-%H-%M")) + '.pkl'
pd.to_pickle(tr_sparse,name_tr_sparse)
pd.to_pickle(te_sparse, name_te_sparse)
